In [ ]:

# Il serait idéal de verifier que d'autres bibliotheques necessitant la version acutelle de python (12 ...) ne seront pas inutilisables apres passage a python 11
# pip freeze > requirements.txt
# Va créer/mettre a jour les dependances 

In [ ]:
# 2 !pip install torch pandas psycopg2


In [ ]:
# 1 Installation des bibliothèques nécessaires
%pip install torch sentence-transformers pandas scikit-learn



Note: you may need to restart the kernel to use updated packages.


In [ ]:
 # 2 Chargement des données du csv 

import pandas as pd
data = pd.read_csv('C:\\Users\\pmgue\\Downloads\\ProjetChefDoeuvre\\RecommendationsLectures\\final_dataset_with_authors_categories.csv')

print(data.head())  # Vérification des données


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1432359641.py, line 4)

In [2]:
# 2 Charger le modèle SentenceTransformer et PyTorch

import torch
from sentence_transformers import SentenceTransformer

# Charger SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Préparer l'utilisation de PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


c:\Users\pmgue\Downloads\ProjetChefDoeuvre\RecommendationsLectures\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [3]:
# 3 Encoder les descriptions en embeddings avec PyTorch

# On augmente la taille maximale des séquences pour traiter toutes les descriptions
model.max_seq_length = 512  # Ajuste si nécessaire pour traiter des textes longs

# Encoder les descriptions en embeddings pour toutes les lignes
embeddings = model.encode(data['description'].fillna(''), convert_to_tensor=True)

# Vérifier si chaque description a bien un embedding généré
if len(embeddings) == len(data['description']):
    print(f"Tous les embeddings ont bien été générés : {len(embeddings)} embeddings pour {len(data['description'])} descriptions.")
else:
    print(f"Problème : {len(embeddings)} embeddings générés pour {len(data['description'])} descriptions.")



Tous les embeddings ont bien été générés : 6481 embeddings pour 6481 descriptions.


In [4]:
# 4 Calculer la similarité cosinus avec PyTorch par batching (lot de 100 ici )

import torch

# Fonction pour calculer la similarité cosinus par lot en utilisant le produit matriciel
def calculate_cosine_similarity_in_batches(embeddings, batch_size=100):
    cosine_sim_list = []
    for i in range(0, embeddings.shape[0], batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        # Calculer la similarité cosinus entre chaque embedding du lot avec produit matriciel
        batch_cosine_sim = torch.mm(batch_embeddings, embeddings.T)  # Produit matriciel
        cosine_sim_list.append(batch_cosine_sim)
    return torch.cat(cosine_sim_list)

# Calculer la similarité cosinus avec batching
cosine_sim_embeddings = calculate_cosine_similarity_in_batches(embeddings, batch_size=100)

# Vérifier la taille de la matrice de similarités cosinus
print(f"Taille des embeddings après calcul de similarité cosinus : {cosine_sim_embeddings.shape}")




Taille des embeddings après calcul de similarité cosinus : torch.Size([6481, 6481])


In [6]:
import torch
# On va enregistrer les embeddings etc pour ne pas avoir a les recalculer a chaque fois
# Enregistrer les embeddings
torch.save(embeddings, 'embeddings.pt')

# Enregistrer la matrice de similarité cosinus
torch.save(cosine_sim_embeddings, 'cosine_sim_embeddings.pt')

print("Embeddings et similarités cosinus sauvegardés.")

Embeddings et similarités cosinus sauvegardés.


In [7]:
# 5 bis Fonction de recommandation de livres sans filtrer par catégorie

# Fonction de recommandation de livres sans filtrer par catégorie
def recommander_livres_sans_categorie(titre_livre, cosine_sim=cosine_sim_embeddings):
    # Recherche du titre du livre (insensible à la casse)
    results = data[data['title'].str.contains(titre_livre, case=False, na=False)]
    
    if results.empty:
        print(f"Le livre '{titre_livre}' n'existe pas dans la base.")
        return []
    
    # Si plusieurs résultats correspondent, prendre le premier
    idx = results.index[0]

    # Calcul des scores de similarité pour le livre recherché
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Trier les livres par similarité, du plus similaire au moins similaire
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Afficher les 3 livres les plus similaires (ignorer le premier qui est le même livre)
    print(f"\nLivres similaires à '{titre_livre}':\n")
    for i, score in sim_scores[1:4]:  # Ignorer le premier (le même livre)
        livre_info = data.iloc[i]
        print(f"Titre : {livre_info['title']}\nAuteur(s) : {livre_info['authors']}\nScore : {score:.2f}\n")

# Exemple de recommandation sans filtrer par catégorie
recommander_livres_sans_categorie("THE NIGHTINGALE")





Livres similaires à 'THE NIGHTINGALE':



KeyError: 'authors'

In [ ]:
# Le score de similarité cosinus est une mesure de similarité entre deux vecteurs. Il peut aller de :

#     -1 : les deux vecteurs pointent dans des directions complètement opposées (très différents).
#     0 : les deux vecteurs sont orthogonaux (pas de similarité).
#     1 : les deux vecteurs pointent dans la même direction (très similaires).

In [ ]:
# Les résultats sans forcer la catégorie sont memes encore meilleurs ce qui est un bon signe que le modèle de similarité cosinus basé sur 
# les descriptions trouve des correspondances pertinentes indépendamment des catégories .

In [ ]:
#            La suite est pour fine-tuner le modele , on reviendra dessus plus tard pour voir si on peut obtenir de meilleurs resultats de cette maniere (GOOGLE COLLAB ?)

In [31]:
print("Taille des embeddings :", cosine_sim_embeddings.shape)
print("Taille des descriptions :", len(data['description']))


Taille des embeddings : torch.Size([6865, 6865])
Taille des descriptions : 6865


In [32]:
# 1 Limiter à la plus petite taille entre les embeddings et les données
num_embeddings = min(cosine_sim_embeddings.shape[0], len(data))

# Préparer des paires d'exemples pour le fine-tuning
train_examples = []
for i in range(num_embeddings):
    for j in range(i + 1, num_embeddings):
        description_1 = data['description'].iloc[i]
        description_2 = data['description'].iloc[j]
        label = cosine_sim_embeddings[i, j].item()  # Similarité cosinus
        train_examples.append(InputExample(texts=[description_1, description_2], label=label))


KeyboardInterrupt: 

In [12]:
# 2 Préparation du Data-Loader et fine-tuning

from torch.utils.data import DataLoader
from sentence_transformers import losses

# Créer un DataLoader pour le fine-tuning
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Définir la perte de similarité cosinus pour l'entraînement
train_loss = losses.CosineSimilarityLoss(model)

# Entraîner le modèle avec PyTorch
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,  # Augmenter le nombre d'époques pour un meilleur résultat
    warmup_steps=100  # Ajuster pour stabiliser l'entraînement
)


NameError: name 'train_examples' is not defined

In [ ]:
# 3: Création du DataLoader et préparation pour le fine-tuning

from torch.utils.data import DataLoader
from sentence_transformers import losses

# Créer un DataLoader pour le fine-tuning
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Définir la perte de similarité cosinus pour l'entraînement
train_loss = losses.CosineSimilarityLoss(model)


In [ ]:


# 4 Entraîner le modèle avec PyTorch
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,  # Ajuster selon les besoins
    warmup_steps=100  # Ajuster pour stabiliser l'entraînement
)


In [ ]:
# 5 Sauvegarder le modèle fine-tuné pour une utilisation ultérieure
model.save('fine_tuned_sentence_transformer')


In [ ]:
# 6 Charger le modèle fine-tuné
model = SentenceTransformer('fine_tuned_sentence_transformer')

# 7 Tester une recommandation
recommander_livres_sans_categorie("THE NIGHTINGALE")
